**VİDEOLAR ÜZERİNDE ÇALIŞMA**

**1-)VİDEO NEDİR?**

Video;film,dizi,hareketli resim,anlık akış kaydetme veya bunların bir ekranda gösterilmesi anlamlarına gelmektedir. **FLİPBOOK**

Videoların oluşumu benzer şekilde resimlerin arka arkaya gösterilmesi denilebilir.Burada akla ilk gelecek soru ne kadar sürede ve ne kadar resmin arka arkaya geleceğidir. **FRAME**

Bir saniye içerisinde gösterilebilecek resim miktarı(Saniye Başına Kare)(Frame Per Second)(FPS) veya FRAME RATE'tir. Performans-Gerçeklik-Harekt/Detay kriterleri vardır.
1. Gerçeklik:Gözleri yormayan bir akıcılık elde etme
2. Hareket:Saniye başına kayıt edilecek kare sayısı ne kadar fazla ise o kadar detayın kaydedilmesi ve yakalanması mümkün olacaktır.
3. Performans:FPS değerlerinin oluşturmak veya görüntülemek için yüksek seviyeli donanım ihtiyacı.

!Yüksek FPS değerleri ile çalışmada donanımın desteklemesi gerekliliği unutulmamalıdır.

**Video dosyaları hem resim karelerini hem de ses dosyasını birlikte senkronize edilmiş biçimde içeren bir bütündür. Bu içeriğin nasıl görüntüleneceği ve depolanacağı ayarları Kodekler ile yapılır.**

OpenCV'de video kayıt etmede FourCC kodeğini kullanabiliriz; DIVX - XVID - H264 - DX50


OpenCV'de bilgisayara bağlı olan bir kameraya ulaşmak veya hazır olan bir video dosyasını açmak aynı işlevdir. **VideoCapture()** sınıfının bir nesnesi oluşturulur.Ardından video dosyasından aldığı videoyu frame frame elde etmek için **read()** metodu oluşturulur.


In [ ]:
import cv2
video = cv2.VideoCapture(index,apiPreference)
video1 = cv2.VideoCapture(filename,apiPreference)
read([,image]) #retval(nesnede çözümlenecek frame olup olmadğına),image(nesneden elde edilmiş frame)
# index:bilgisayara bağlı kameranın indeks numarası 0 olursa varsayılan kamera(Varsayılan:None) - filename:video dosyasının adres yolu(Varsayılan:None)
# apiPreference:Kullanıcının erişemediği alana ulaşmak için kullanılır.

In [2]:
import cv2

camera = cv2.VideoCapture(0)

if camera.isOpened() == False:
    print("Doğru gitmeyen bir şeyler var.")
    
while camera.isOpened:
    ret,frame = camera.read()
    
    if ret == True:
        cv2.imshow("Video Dosya Okuma",frame)
        
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break
            
    else:
        break
        
camera.release()
cv2.destroyAllWindows()

Video kayıt etmek için **VideoWriter()** Herhangi bir parametre sağlanmadan video dosyasının oluşturulması mümkün değildir.

In [ ]:
import cv2
camera_save = cv2.VideoWriter()
cameraSave = cv2.VideoWriter(filename,fourcc,fps,frameSize[,isColor])
Camerasave = cv2.VideoWriter(filename,apiPreference,fourcc,fps,frameSize[,isColor])

In [1]:
import time
import cv2

video = cv2.VideoCapture("OneDrive/Belgeler/OpenCV/Kitap/DATA/videoplayback.mp4")

width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

if video.isOpened() == False:
    print("Doğru gitmeyen bir şeyler var.")
    
vw = cv2.VideoWriter('videoplay_gray.mp4',cv2.VideoWriter_fourcc(*'XVID'),fps,(width,height),isColor=False)

while video.isOpened:
    ret,frame = video.read()
    
    if ret == True:
        time.sleep(1 / fps)
        frame = cv2.circle(frame,(55,55),45,color=(255,0,0),thickness=-1)
        frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        cv2.imshow('Video Dosya Okuma',frame_gray)
        vw.write(frame_gray)
        
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break
        
        
    else:
        break
        
vw.release()
video.release()
cv2.destroyAllWindows()

print(width)
print(height)
print(fps)

640
360
25.0


**!Gerekli durumlarda VideoCapture() sınıfı ile birlikte bir resim sıralaması giriş parametresi olarak sağlanabilir. Bunun için tek gerekli olan parametre olarak düzgün isimlendirme yapılmalıdır. Örneğin img%02.jpg şeklinde girilecek bir parametre img00.jpg,img01.jpg,img02.jpg... şeklinde oluşturulan bir nesneden get() metodu ile alınacak FPS bilgisinin 0 çıkacağı unutulmamalıdır.**

**2-)OPTİCAL FLOW(Optik Akış)**

Nesnenin veya kameranın hareketinden kaynaklı iki ardışık çerçeve(frame) arasındaki görünür hareket modelidir.Yani görüntüde nesnelerin nasıl hareket ettiğinin tespit edilmesi işlemidir.Bunun için ardışık iki zaman çerçevesi arasındaki piksel konumunun değişimi hesaplanır.

(x,y)&(dx,dy)  -----> (x+dx,y+dx)
 frame_001(t)          frame_002(t + dt)
 
Optik Akışın Varsayımları:

1. Bir nesnenin renk yoğunluğu ve piksel yoğunlukları ardışık kareler arasında önemli ölçüde değişmez.
2. Görüntü düzlemindeki birbirine yakın noktalar benzer şekilde hareket eder.
(Gerçek hayatta nerede ise imkansızdır.)

Görüntü dengeleme ve görüntü sabitlemede kullanılır.

**Lucas-Kanade Method**

Bu yöntemde belirlenen bir nokta etrafında 3X3'lük piksel kesit alınarak takibi gerçekleştirir.Büyük boyutta olan hareketlerin algılanmasını resim büyüklüğünü küçülterek hareketi de küçültmeye teşvik etmiştir.Amaç görüntüdeki bütün büyük hareketlerin yakalanması mümkün olabilir.

OpenCV'de **calcOpticalFlowPyrLK()** fonksiyonu kullanılmaktadır. Öncelikle köşe tespit algoritması kullanılır ardından hareket modeli çıkarılabilir.


In [ ]:
import cv2

status = cv2.calcOpticalFlowPyrLK(prevImg,nextImg,prevPts,nextPts[,status[,err[,winSize[,maxLevel[,criteria[,flags[,minEigThreshold]]]]]]])
# prevImg:ilk piramit yapısı - nextImg:ikinci piramit yapısı - prevPts:ilk görüntüde akışın hesaplanacağı koordinat - winSize:taramadan kullanılcak parçacık boyutu(Varsayılan:21,21)
# maxLevel:piramit seviye sınırı - criteria:sonlanma kriterleri(TermCRİTERİA) - flags:çalışma biçimi((0),4,8,256)

criteria = cv2.TERM_CRITERIA_COUNT + cv2.TERM_CRITERIA_EPS


In [ ]:
import numpy as np
import cv2

cornerDetectionCriteria = dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)
lucasCriteria = dict(winSize=(200,200),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT,10,0.03))

camera = cv2.VideoCapture(0)

ret, prevFrame = camera.read()
grayPrevFrame = cv2.cvtColor(prevFrame,cv2.COLOR_BGR2GRAY)

prevPoints = cv2.goodFeaturesToTrack(grayPrevFrame,mask=None,**cornerDetectionCriteria)
mask = np.zeros_like(prevFrame)

if camera.isOpened() == False:
    print("Doğru gitmeyen bir şeyler var.")
    
while camera.isOpened:
    ret,currentFrame = camera.read()
    grayCurrentFrame = cv2.cvtColor(currentFrame,cv2.COLOR_BGR2GRAY)
    
    nextPoints,status,errors = cv2.calcOpticalFlowPyrLK(grayPrevFrame,grayCurrentFrame,prevPoints,None,**lucasCriteria)
    
    newmotion = nextPoints[status == 1].reshape(-1,2)
    oldmotion = prevPoints[status == 1].reshape(-1,2)
    
    for i,(new,prev) in enumerate(zip(newmotion,oldmotion)):
        x_newPos,y_newPos = new
        x_prevPos,y_prevPos = prev
        
        pt1 = (int(x_newPos), int(y_newPos))
        pt2 = (int(x_prevPos), int(y_prevPos))
        
        mask = cv2.line(mask,pt1,pt2,(0,255,0),2)
        currentFrame = cv2.circle(currentFrame,pt1,8,(0,0,255),-1)
        
        img = cv2.add(currentFrame,mask)
        cv2.imshow('takip',img)
        
        k = cv2.waitKey(10) & 0XFF
        if k == 27:
            break
            
        grayPrevFrame = grayCurrentFrame.copy()
        nextPoints = newmotion.reshape(-1,1,2)
        
cv2.destroyAllWindows()
camera.release()

**Derin Oprik Akış Gunner-FarnerBack Algoritması**

İlk adım **POLİNOM AÇILIMI**'dır.Her bir pikselin ikinci dereceden polinomları hesaplanarak yakınlaştırılmaktadır.

İkinci adım **YER DEĞİŞİMİ TAHMİNİ**'dir.polinominal olarak tahmin edilen her bir komşunun,ötelenmesi kaydırılması sonucu analiz edilir.

Üçüncü adım **KARŞILAŞTIRMA**'dır.

Renk uzayını **HSV**'den feyiz alır.

OpenCV'de **calcOpticalFarneback()**

In [ ]:
import cv2
flow = cv2.calcOpticalFlowFarneback(prev,next,flow,pyr_scale,levels,winsize,iterations,poly_n,poly_sigma,flags)
# flow:CV_32FC2 tipinde ilk çerçeve ile aynı boyutta hesaplanmış çıkış akış görünütüsü
# poly_n:her piksel için polinimal genişlemeyi hesaplamak için kullanılan komşu piksel sayısıdır.
# poly_sigma:polinom açılımında kullanılacak türevleri yumuşatmak için gerekli olan gaussian standart sapması değeridir.
# poly_n=5,poly_sigma=1.1 -- poly_n=7,poly_sigma=1.5

**calcOpticalFlowFarneback()** çalıştırıldıktan sonra geriye optik akış hesaplamasının yapılmış olduğu iki boyutlu bir vektör döndürmektedir.Ancak bu fonksiyon ile elde edilen sonuçların görselleştirilmesi yani akış tespiti yapılan piksellerin gösterilebilmesi için elde edilen bu iki boyutlu listede vektör büyüklüklerinin ve açılarının hesaplanması gerekmektedir.

OpenCV'de **cartToPolar()** fonksiyonu

In [ ]:
import cv2
magnitude,angle = cv2.cartToPolar(x,y[,magnitude[,angle[,angleInDegrees]]])

# x,y:x ve y ekseni bilgileri sağlanır - 
# magnitude:x parametresiyle aynı boyutta hesaplanılmış çıktının kare farkının kök içerisinde alınmasıdır.
# angle:x parametresiyle aynı boyutta hesaplanılmış çıktının radyan ya da derece cinsinden açı bilgisinin alınmasıdır. 

In [ ]:
import numpy as np
import cv2

video = cv2.VideoCapture(0)

ret,prevFrame = video.read()
grayPrevFrame = cv2.cvtColor(prevFrame,cv2.COLOR_BGR2GRAY)

hsvMask = np.zeros_like(prevFrame)
hsvMask[:,:,1] = 255

while True:
    ret,nextFrame = video.read()
    grayNextFrame = cv2.cvtColor(nextFrame,cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(grayPrevFrame,grayNextFrame,None,0.5,3,15,3,5,1.1,0)
    
    magnitude,angle = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    hsvMask[:,:,0] = angle / 2
    
    hsvMask[:,:,2] = cv2.normalize(magnitude,None,0,255,cv2.NORM_MINMAX)
    
    bgrMask = cv2.cvtColor(hsvMask,cv2.COLOR_HSV2BGR)
    
    cv2.imshow('Dense OF',bgrMask)
    
    if cv2.waitKey(10) & 0XFF == 27:
        break
        
    prevFrame = nextFrame
    
video.release()
cv2.destroyAllWindows()

**3-)NESNE TAKİP ALGORİTMALARI**

Nesne tespiti yapıldıktan sonra aynı şekilde ardışık frame'de kendisini tekraren takibini yapmak önem arz etmektedir.Ayrıca bu işlem o kadar hızlı olmalı ki bu işlem kendini süregelen bir şekilde tekrar edilsin.Hem fiziksel hem de soyutsal değişkenlerin baş göstermesi ile Nesne Takibi bilgisayarlı görü alanın en zor iş parçacık kısmıdır.Nesne Takip Algoritmaları Adımları:

1. Takibi yapılacak nesnenin belirlenmesi
2. Takip edilecek nesne için özellik çıkarma yöntemleri veya örüntü tanıma ile nesneye özel benzersiz bir kimlik oluşturulması.
3. Video'da her ardışık çerçevede tanımlanan kimliğin tekrar edilmesi.

OpenCV'de **BOOSTING,MIL,KCF,CSRT,TLD,MedianFlow,MOSSE VE GOTURN** optimize edilen hareket izleminin nesne takip algoritmaları bulunmaktadır.Algoritmaları seçer iken bu hususlar dikkate almalıyız;

1. Üzerinde çalışılacak görüntünün hazır bir video dosyasından alınıp/alınmadığı, canlı video akışı olup/olmadığı veya sabit görüntülerden oluşup/oluşmadığı
2. Kullanım alanı ve amacı
3. Beklenen arka plan gürültüleri ve seviyeleri
4. Algılandıktan sonra takibi yapılacak nesnenin farklı boyutlarının bulunması
5. Aydınlığın değişme oranı
6. Birbiriyle ters orantılı olan hız ve doğruluğun önem dereceleri

**Nesne tespit bir görüntüde bulunan nesnenin tanımlanması işlemidir, nesne takip ise herhangi bir nesnenin bir frameden/çerçeveden diğerine değişimini izlemek ve işaretlemektir.Genellikle takip algoritmaları ,tespit algoritmalarından daha hızlıdır.**

Aslında görüntü işlemede piksel yoğunluklarına göre kümeleme işlemi yapılacaktır.

**Nesne takip uygulamalarında dikkat edilmesi gereken en önemli husus;Takip edilen nesnenin objektifinden ayrılması veya önüne başka engel geçmesiyle birden ortadan kaybolmasıdır.Bu sebeple iyi geliştirilen bir uygulamada bu tür durumlar düşünülmeli,planlanmalı ve başarılı bir şekilde ele alınmalıdır.**

**meanShift & camShift Algoritması**

meanShift:Kendisine verilen bir pencere içerisindeki piksel yoğunluklarına bakarak yoğunluğu en fazla olduğu alana doğru kendi penceresini kaydırmasıdır.Görüntü üzerinde belirlenen bir alanın histogram eğrisi ve piksel yoğunluğu çıkartılarak, sonradaki çerçevede taramak için kullanılır.

Benzersiz bir model oluşturulması için **calcBackProject()** kullanmak bir görüntüdeki piksellerin bir histogramdaki piksel dağılımına göre ne kadar uyumlu olduğunu belirleme ve kayıt etme.Bu fonksiyon ile elde edilen projeye **backProjection** isimlendirilmesi yapılmaktadır bir nevi geri yayılım.Bu hesaplamada 180 derece baz alınır,diğer 180 derece yansıma bazıdır.Histogram eğrisini baz almak için önce HSV renk uzayına ardından normalize etmek gereklidir.

In [ ]:
import cv2
retval,window = cv2.meanShift(probImage,window,criteria)
retval1,window1 = cv2.CamShift(probImage,window,criteria)
target = cv2.calcBackProject(images,channels,hist,ranges,scale[,dst])

In [1]:
import numpy as np
import cv2

video = cv2.VideoCapture(0)
ret,prevFrame = video.read()

faceDetection = cv2.CascadeClassifier('OneDrive/Belgeler/OpenCV/Kitap/DATA/haarcascades/haarcascade_frontalface_default.xml')
faceLocation = faceDetection.detectMultiScale(prevFrame)

if len(faceLocation)>0:
    (faceX,faceY,w,h) = tuple(faceLocation[0])
    faceTracking =(faceX,faceY,w,h)

    roi = prevFrame[faceY:faceY + h,  faceX:faceX + w]
    hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV) 

    hist_roi = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])
    cv2.normalize(hist_roi,hist_roi,0,255,cv2.NORM_MINMAX)

    parameters = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT , 10 , 1)

    while True:
        ret,nextFrame = video.read()
        hsv_NextFrame = cv2.cvtColor(nextFrame,cv2.COLOR_BGR2HSV)

        target = cv2.calcBackProject([hsv_NextFrame],[0],hist_roi,[0,180],1)
        ret,faceTracking = cv2.meanShift(target,faceTracking,parameters)

        faceX,faceY,w,h = faceTracking

        mark = cv2.rectangle(nextFrame,(faceX,faceY),(faceX + w ,faceY + h ),(0,255,255),5)
        cv2.imshow('Takip',mark)

        k = cv2.waitKey(10) & 0XFF

        if k == ord('q'):
            break           
else:
    print("DONT FACE DETECTION!")
        
video.release()
cv2.destroyAllWindows()

Kameraya doğru yaklaşan bir aracın görüntüsünde aracın boyutu ve açısı sürekli olarak değişmesine rağmen meanShift ile yapılan nesne takibinde nesne takip penceresi bu değişime göre uyarlanamayacak ve sabit boyutu kalacaktır.Buna çözüm ise **CamShift()** algoritmasıdır.(Continuously Adaptive MeanShift).

Öncelikle meanShift algoritması kullanılarak nesne kaymasının veya takibin olduğu konumu hesaplatmaktadır sonrasında algoritmasının her iterasyonunda hesaplanan aynı konum için en ideal elips ölçüsü ve açısının hesaplamasını gerçekleştirir.Bu sayede ardışık framelerde/çerçevelerde yakalanan her hareketi gösteren takip penceresinin açısı ve boyutu güncellenmiş olacaktır.

In [4]:
import numpy as np
import cv2

video = cv2.VideoCapture(0)
ret,prevFrame = video.read()

faceDetection = cv2.CascadeClassifier('OneDrive/Belgeler/OpenCV/Kitap/DATA/haarcascades/haarcascade_frontalface_default.xml')
faceLocation = faceDetection.detectMultiScale(prevFrame)
    
(faceX, faceY,w, h) = tuple(faceLocation[0])
faceTracking = (faceX, faceY, w, h)

roi = prevFrame[faceY : faceY + h, faceX : faceX + w]
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

hist_roi = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])
cv2.normalize(hist_roi,hist_roi,0,255,cv2.NORM_MINMAX)

parameters = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT , 10 ,1)

while True:
    ret,nextFrame = video.read()
    hsv_NextFrame = cv2.cvtColor(nextFrame,cv2.COLOR_BGR2HSV)

    target = cv2.calcBackProject([hsv_NextFrame],[0],hist_roi,[0,180],1)
    ret,faceTracking = cv2.CamShift(target,faceTracking,parameters)

    points = cv2.boxPoints(ret)
    points = np.int0(points)

    mark = cv2.polylines(nextFrame,[points],True,(0,255,255),5)

    cv2.imshow('Takip',mark)

    k = cv2.waitKey(10) & 0XFF

    if k == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

C:\Users\emred\AppData\Local\Temp\ipykernel_7788\2078378758.py:29: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  points = np.int0(points)


**meanShift()** & **CamShift()** algoritmaları ilk olarak yüz tespiti yapılmış sonra ise takibi yapılmıştır.Ancak bu uygulamalar geliştirilerek her türlü nesnenin takibi için kullanılması mümkün olabilmektedir.Öncelikle nesnenin konumuna ve boyutuna bir şekilde ulaşılması gerekliliğidir.Sonrasında meanshift ve camshift ile nesne takibi gerçekleştirilebilmektedir.

Bu algoritmaların temel amacı renk yoğunluğu veya dağılımının tespit edilmesi olup nesne takip amaçlı değildir.Ancak ardışık frameler/çerçeveler arasında aynı yoğunluğu takip edebilmesinden ötürü nesne takip uygulamaları kullanılabilmektedir.

Eğer sistem önce bir veri seti ile eğitilerek,eğitilmiş olduğu nesnenin tespiti ve takibi yaptılırsa off-line,eğer sistem anlık almış olduğu veri seti ile eğitilirse on-line.

In [8]:
import sys
import cv2
import numpy as np

def trackerSelection():
    print("Enter 1 for MIL,2 for GOTURN")
    choice = input("Please choice your Tracker?")
    
    if choice == '1':
        tracker = cv2.TrackerMIL()
    elif choice == '2':
        tracker = cv2.TrackerGOTURN()
    return tracker

tracker = trackerSelection()
trackerName = str(tracker).split()[0][1:]
camera = cv2.VideoCapture("OneDrive/Belgeler/OpenCV/Kitap/DATA/videoplayback_4.mp4")
ret,frame = camera.read()

roi = cv2.selectROI(frame,False)
ret = tracker.init(frame,roi)

while True:
    ret,frame = camera.read()
    success,roi = tracker.update(frame)
    
    (x,y,w,h) = tuple(map(int,roi))
    
    if success:
        p1 = (x,y)
        p2 = (x + w , y + h)
        cv2.rectangle(frame,p1,p2,(0,255,0),3)
    else:
        cv2.putText(frame,"Failure!",(100,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        cv2.putText(frame,trackername,(20,400),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        cv2.imshow(trackerName,frame)
        
        k = cv2.waitKey(10) & 0XFF
        if k == 27:
            break
            
camera.release()
cv2.destroyAllWindows()

Enter 1 for MIL,2 for GOTURN


Please choice your Tracker? 2


error: Unknown C++ exception from OpenCV code